In [0]:
# # ruta="/content/drive/My Drive/Noticias/Alertas Negativas/"
# ruta="/content/drive/My Drive/Alertas Negativas/"

In [0]:
!pip install -q twitterscraper

In [0]:
import twitterscraper
from twitterscraper.query import query_tweets #obtener todos los tweets con una palabra clave
import datetime as dt ## para el manejo de las fechas
import time ## para calcular el tiempo de procesamiento
import pandas as pd ### para el manejo de datframes
import numpy as np

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Alertas_Negativas_Twitter/notebooks")

### Scrapear información de los tuits

In [0]:
today = dt.date.today()
tomorrow = dt.date.today() + dt.timedelta(days=1)

def get_tweet(query,limit=None,begindate=today,enddate=tomorrow,poolsize=20,lang=" "):
  list_tweet=[]
  if enddate != tomorrow:
    enddate=enddate + dt.timedelta(days=1)
  list_text=query_tweets(query, limit=limit,begindate=begindate ,enddate=enddate, poolsize=poolsize, lang=lang) ## es 
  hour_per = dt.timezone(dt.timedelta(hours=-5))#Configurar la zona horaria 
  for tweet in list_text:
        dict_ = {# user name & id
                "screen_name":tweet.screen_name,
                "username":tweet.username,
                "user_id":tweet.user_id,
                # tweet basic data
                "tweet_id":tweet.tweet_id,
                "tweet_url":tweet.tweet_url,
                "timestamp":tweet.timestamp.astimezone(hour_per),
                "timestamp_epochs":tweet.timestamp_epochs,
                # tweet text
                "text":tweet.text,
                "text_html":tweet.text_html,
                "links":tweet.links,
                "hashtags":tweet.hashtags,
                # tweet media
                "has_media":tweet.has_media,
                "img_urls":tweet.img_urls,
                "video_url":tweet.video_url,
                # tweet actions numbers
                "likes":tweet.likes,
                "retweets":tweet.retweets,
                "replies":tweet.replies,
                "is_replied":tweet.is_replied,
                # detail of reply to others
                "is_reply_to":tweet.is_reply_to,
                "parent_tweet_id":tweet.parent_tweet_id,
                "reply_to_users":tweet.reply_to_users
                }
        list_tweet.append(dict_)
  df=pd.DataFrame(list_tweet).sort_values(by="timestamp",ascending=False)
  df.drop_duplicates(subset=["user_id","tweet_id"],keep="first",inplace=True)
  df.index=[i for i in range(0,len(df))]
  return df

In [0]:
tomorrow = dt.date.today() + dt.timedelta(days=1)
df_tweets_total=get_tweet(query="@bcrpoficial",begindate=dt.date(2020,1, 18),enddate=tomorrow,poolsize=150)
display(df_tweets_total)

In [0]:
df_tweets_total.to_pickle("../data/raw/df_tweets_total.pkl")

In [0]:
# df_tweets_total=pd.read_pickle("../data/raw/df_tweets_total.pkl")

##### Obtener todas las publicaciones realizadas por el banco


In [30]:
aux=df_tweets_total.groupby(["parent_tweet_id"]).tweet_id.count().to_frame().rename(columns={"tweet_id":"comment_total"})
df_tweets_banco=df_tweets_total[(df_tweets_total["screen_name"]=="bcrpoficial") & (df_tweets_total["parent_tweet_id"]=="")]
df_tweets_banco.index=[ i for i in range(0,len(df_tweets_banco))]
df_tweets_banco=pd.merge(df_tweets_banco, aux ,how="left",left_on="tweet_id",right_on="parent_tweet_id")
df_tweets_banco["comment_total"]=df_tweets_banco["comment_total"].fillna(value=0).astype("int32")
display(df_tweets_banco)

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,hashtags,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users,comment_total
0,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1218971489208238081,/bcrpoficial/status/1218971489208238081,2020-01-19 14:00:25-05:00,1579460425,Descarga gratuitamente en PDF los libros del C...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://www.bcrp.gob.pe/publicaciones/compendi...,[],True,[https://pbs.twimg.com/media/EOqoG_TW4AANPe3.jpg],,4,1,0,False,False,,[],0
1,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1218956404347887616,/bcrpoficial/status/1218956404347887616,2020-01-19 13:00:28-05:00,1579456828,El #BCRP amplió los plazos de colocación de su...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[BCRP],True,[https://pbs.twimg.com/media/EOqaY6XXUAILbx9.jpg],,2,1,0,False,False,,[],0
2,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1218941320074493959,/bcrpoficial/status/1218941320074493959,2020-01-19 12:00:32-05:00,1579453232,Nota de Estudios del #BCRP: Operaciones del Se...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://www.bcrp.gob.pe/docs/Publicaciones/Not...,[BCRP],True,[https://pbs.twimg.com/media/EOqMq8PWAAY7PUP.jpg],,2,0,0,False,False,,[],0
3,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1218926193690451968,/bcrpoficial/status/1218926193690451968,2020-01-19 11:00:25-05:00,1579449625,#BCRP: Los puestos de trabajo formales en el s...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://www.bcrp.gob.pe/docs/Publicaciones/Not...,[BCRP],True,[https://pbs.twimg.com/media/EOp-6bxXkAAoz7U.jpg],,11,2,0,False,False,,[],0
4,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1218911120544518144,/bcrpoficial/status/1218911120544518144,2020-01-19 10:00:32-05:00,1579446032,"#BCRP: Recaudación del IGV creció 4,7% en 2019...","<p class=""TweetTextSize js-tweet-text tweet-te...",[http://www.bcrp.gob.pe/docs/Publicaciones/Not...,[BCRP],True,[https://pbs.twimg.com/media/EOpxNFYXkAEW4l8.jpg],,5,1,0,False,False,,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,3980344286,/bcrpoficial/status/3980344286,2009-09-14 08:57:03-05:00,1252936623,I uploaded a YouTube video -- Canje de billete...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,False,,[],0
8549,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,3979847502,/bcrpoficial/status/3979847502,2009-09-14 08:26:36-05:00,1252934796,I uploaded a YouTube video -- Cuidado del bill...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,False,,[],0
8550,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,3912228948,/bcrpoficial/status/3912228948,2009-09-11 09:44:59-05:00,1252680299,I uploaded a YouTube video -- Medidas de segur...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,False,,[],0
8551,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,3911787514,/bcrpoficial/status/3911787514,2009-09-11 09:21:26-05:00,1252678886,I uploaded a YouTube video -- v1 700 http://bi...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,False,,[],0


In [0]:
df_tweets_banco.to_pickle("../data/raw/df_tweets_banco.pkl")

##### obtener todas las respuestas del Banco

In [32]:
df_tweets_banco_repply=df_tweets_total[(df_tweets_total["screen_name"]=="bcrpoficial") & (df_tweets_total["parent_tweet_id"]!="")]
print(df_tweets_banco_repply.shape)
df_tweets_banco_repply.head()

(57, 21)


,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,hashtags,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users
121,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1216728465921257472,/bcrpoficial/status/1216728465921257472,2020-01-13 09:27:26-05:00,1578925646,Gracias por tu interés. Nunca se emitió billet...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://www.bcrp.gob.pe/publicaciones/libro-bi...,[],False,[],,1,0,0,False,True,1216696779141533696,"[{'screen_name': 'shalyjr', 'user_id': '154370..."
3454,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1187488669923495936,/bcrpoficial/status/1187488669923495936,2019-10-24 16:58:56-05:00,1571954336,Este comentario alude a Argentina. @bcrpoficia...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,1,0,0,False,True,1187482314974126080,"[{'screen_name': 'cyngarciaradio', 'user_id': ..."
3455,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1187488406177308677,/bcrpoficial/status/1187488406177308677,2019-10-24 16:57:53-05:00,1571954273,Este comentario alude a Argentina. @bcrpoficia...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,1,True,True,1187482314974126080,"[{'screen_name': 'cyngarciaradio', 'user_id': ..."
3456,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1187487949149130758,/bcrpoficial/status/1187487949149130758,2019-10-24 16:56:04-05:00,1571954164,Este comentario alude a Argentina. @bcrpoficia...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,True,1187482314974126080,"[{'screen_name': 'cyngarciaradio', 'user_id': ..."
12793,bcrpoficial,Banco Central de Reserva del Perú - BCRP,70469669,1068270174795689985,/bcrpoficial/status/1068270174795689985,2018-11-29 17:27:11-05:00,1543530431,"El presidente del @bcrpoficial, Julio Velarde,...","<p class=""TweetTextSize js-tweet-text tweet-te...",[https://twitter.com/noticias_tvperu/status/10...,[],False,[],,0,0,0,False,True,1068205414926479361,"[{'screen_name': 'LBHobsbawm', 'user_id': '151..."


In [0]:
df_tweets_banco_repply.to_pickle("../data/raw/df_tweets_banco_repply.pkl")

##### Obtener publicaciones que mencionan al banco 

In [34]:
### Obtener todas las menciones al Banco Central
df_tweets_mention=df_tweets_total[(df_tweets_total["screen_name"]!="bcrpoficial") & (df_tweets_total["parent_tweet_id"]=="")]
df_tweets_mention.index=[ i for i in range(0,len(df_tweets_mention))]
df_tweets_mention=pd.merge(df_tweets_mention, aux ,how="left",left_on="tweet_id",right_on="parent_tweet_id")
df_tweets_mention["comment_total"]=df_tweets_mention["comment_total"].fillna(value=0).astype("int32")
display(df_tweets_mention)

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,hashtags,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users,comment_total
0,mister_cejas,Olaechea Disuelto,1119748332337942528,1218451322846031872,/mister_cejas/status/1218451322846031872,2020-01-18 03:33:27-05:00,1579336407,Será un indicio para llegar este 2020 al 4% de...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[https://twitter.com/ToniAlvaL/status/12183480...,[],False,[],,1,0,0,False,False,,[],0
1,CamaraCaj,Cámara de Comercio y Producción de Cajamarca,1129411683858960384,1218319694719082496,/CamaraCaj/status/1218319694719082496,2020-01-17 18:50:25-05:00,1579305025,@CamaraCaj y @bcrpoficial presentan Conferenc...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[https://bit.ly/376jGcO],[],True,[https://pbs.twimg.com/media/EOhXQl9X0AMWQeO.jpg],,0,0,0,False,False,,[],0
2,DiarioElPeruano,Diario El Peruano,3957668722,1218286119646846978,/DiarioElPeruano/status/1218286119646846978,2020-01-17 16:37:00-05:00,1579297020,El déficit fiscal se redujo de 2.3% del produ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://bit.ly/2RrXi7d],[],True,[https://pbs.twimg.com/media/EOgvWAkX4AEe_tU.jpg],,1,2,0,False,False,,[],0
3,Agencia_Andina,Agencia Andina,40933920,1218256675657830400,/Agencia_Andina/status/1218256675657830400,2020-01-17 14:40:00-05:00,1579290000,Banco Central de Reserva (@bcrpoficial): recau...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://bit.ly/2TFE1BM],[],True,[https://pbs.twimg.com/media/EOgbTBEXUAAAt3_.jpg],,0,2,0,False,False,,[],0
4,sientetrujillo,Siente Trujillo,567366294,1218199511178383361,/sientetrujillo/status/1218199511178383361,2020-01-17 10:52:51-05:00,1579276371,#SienteTrujillo: Minería y agricultura son ej...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://bit.ly/2RpLItk],[SienteTrujillo],False,[],,1,1,0,False,False,,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10116,JamesTheGorilla,Julio IsЯael,104971262,102005048921440256,/JamesTheGorilla/status/102005048921440256,2011-08-12 08:14:33-05:00,1313154873,#FF @YoDesinformo @juanjo_cas @RubiaYFuriosa @...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[FF],False,[],,0,0,0,False,False,,[],0
10117,Peru_Finance,Perú Finance,203387985,86494800634843136,/Peru_Finance/status/86494800634843136,2011-06-30 13:02:22-05:00,1309456942,RT @bcrpoficial: Nuevos Billetes de S/. 10 y S...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://www.facebook.com/pages/Banco-Central-d...,"[BCRP, Peru]",False,[],,0,0,0,False,False,,[],0
10118,JamesTheGorilla,Julio IsЯael,104971262,66520997565710337,/JamesTheGorilla/status/66520997565710337,2011-05-06 10:13:36-05:00,1304694816,#FF @Arianahot @OfficialOzzy @ElOtorongo @raga...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[FF],False,[],,0,0,0,False,False,,[],0
10119,JamesTheGorilla,Julio IsЯael,104971262,63987584128393217,/JamesTheGorilla/status/63987584128393217,2011-04-29 10:26:43-05:00,1304090803,#FF a @Arianahot @Rockmanita @Korn @slipknot @...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[FF],False,[],,0,0,0,False,False,,[],0


In [0]:
df_tweets_mention.to_pickle("../data/raw/df_tweets_mention.pkl")

##### Obtener tuits que responden a publicaciones , ya sea del Banco o de terceros

In [36]:
df_tweets_repply=df_tweets_total[(df_tweets_total["screen_name"]!="bcrpoficial") & (df_tweets_total["parent_tweet_id"]!="")]
df_tweets_repply.index=[ i for i in range(0,len(df_tweets_repply))]
display(df_tweets_repply)

,screen_name,username,user_id,tweet_id,tweet_url,timestamp,timestamp_epochs,text,text_html,links,hashtags,has_media,img_urls,video_url,likes,retweets,replies,is_replied,is_reply_to,parent_tweet_id,reply_to_users
0,MARTELMILTON,MILTON MARTEL,452727740,1218539475141300224,/MARTELMILTON/status/1218539475141300224,2020-01-18 09:23:44-05:00,1579357424,El @JNE_Peru tiene que pronunciarse ya..\nSin ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,1,0,0,False,True,1218368404660346890,"[{'screen_name': 'reyreysincorona', 'user_id':..."
1,Mercede39096338,Mercedes Mendoza,883189569801289729,1218413412549525504,/Mercede39096338/status/1218413412549525504,2020-01-18 01:02:49-05:00,1579327369,,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,True,1218398876555759617,"[{'screen_name': 'bcrpoficial', 'user_id': '70..."
2,FelizDiaLima,#FelizDíaLima. 👑👑👑⭐️,1218063509549993986,1218399453335498752,/FelizDiaLima/status/1218399453335498752,2020-01-18 00:07:21-05:00,1579324041,Amamos nuestra ciudad. #FelizDíaLima,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[FelizDíaLima],False,[],,0,0,0,False,True,1218398876555759617,"[{'screen_name': 'bcrpoficial', 'user_id': '70..."
3,calamaro_milton,Milton_Calamaro,1187444833549541376,1218307841624289282,/calamaro_milton/status/1218307841624289282,2020-01-17 18:03:19-05:00,1579302199,aaaaaaaaaaaaaaaaaaaaaaa,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,True,1217184674587586565,"[{'screen_name': 'bcrpoficial', 'user_id': '70..."
4,CastroJcalin,BrahMos,2544561561,1218300429538414593,/CastroJcalin/status/1218300429538414593,2020-01-17 17:33:51-05:00,1579300431,Lo que me extraña es porque BCP aún no se une ...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,4,0,0,False,True,1218183446167724032,"[{'screen_name': 'KeroPeru', 'user_id': '91462..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18733,JamesTheGorilla,Julio IsЯael,104971262,86891224103002112,/JamesTheGorilla/status/86891224103002112,2011-07-01 15:17:36-05:00,1309551456,@bcrpoficial #Fail @perufail Nuevo Post: Nuevo...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[http://bit.ly/lEEvci],[Fail],False,[],,0,0,0,False,True,86890477571407873,"[{'screen_name': 'perufail', 'user_id': '14252..."
18734,JamesTheGorilla,Julio IsЯael,104971262,86861868672622592,/JamesTheGorilla/status/86861868672622592,2011-07-01 13:20:57-05:00,1309544457,@ErvaVenenosaa Pero se espera q a fin de ano c...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,1,0,False,True,86858086865833984,"[{'screen_name': 'FaRubioV', 'user_id': '95826..."
18735,willy750729,Willy Ramírez,88739200,86781684997033985,/willy750729/status/86781684997033985,2011-07-01 08:02:20-05:00,1309525340,@bcrpoficial Les sugiero aclarar que el revers...,"<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,True,86487513610465281,"[{'screen_name': 'bcrpoficial', 'user_id': '70..."
18736,jomisilvamerino,José Silva,123374537,86483115958214656,/jomisilvamerino/status/86483115958214656,2011-06-30 12:15:56-05:00,1309454156,"@bcrpoficial amigos, a nosotros también envíen...","<p class=""TweetTextSize js-tweet-text tweet-te...",[],[],False,[],,0,0,0,False,True,86482622632558592,"[{'screen_name': 'bcrpoficial', 'user_id': '70..."


In [0]:
df_tweets_repply.to_pickle("../data/raw/df_tweets_repply.pkl")